# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.53,88,90,3.25,TF,1699950823
1,1,college,64.8569,-147.8028,-3.63,92,98,1.22,US,1699950824
2,2,inarajan village,13.2736,144.7484,29.08,89,81,9.48,GU,1699950825
3,3,kirensk,57.7853,108.1119,-0.02,83,100,5.81,RU,1699950826
4,4,cantley,53.5000,-1.0500,10.43,90,86,3.20,GB,1699950827


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    size='Humidity',
    title='World Map with Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    hover_cols=['City', 'Humidity'],
    geo=True,
    tiles='OSM',
    alpha=0.5,  
    cmap='viridis', 
    clim=(city_data_df['Humidity'].min(), city_data_df['Humidity'].max()),
    colorbar=True,
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,balsas,-7.5325,-46.0356,24.37,56,0,0.94,BR,1699950840
39,39,san blas,21.5167,-105.2667,24.65,71,0,1.06,MX,1699950859
192,192,tan-tan,28.4380,-11.1032,25.97,27,0,3.09,MA,1699950992
232,232,waddan,29.1614,16.1390,21.10,62,0,3.34,LY,1699951030
283,283,balkanabat,39.5108,54.3671,26.10,18,0,4.47,TM,1699951074
313,313,parauapebas,-6.0675,-49.9022,23.42,63,0,0.28,BR,1699950963
317,317,remire-montjoly,4.9167,-52.2667,24.02,94,0,0.00,GF,1699951103
336,336,ghat,24.9647,10.1728,21.77,38,0,2.73,LY,1699951120
365,365,san sebastian de la gomera,28.0916,-17.1133,24.20,49,0,1.03,ES,1699951144
477,477,floriano,-6.7669,-43.0225,24.34,62,0,2.22,BR,1699951237


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,balsas,BR,-7.5325,-46.0356,56,
39,san blas,MX,21.5167,-105.2667,71,
192,tan-tan,MA,28.4380,-11.1032,27,
232,waddan,LY,29.1614,16.1390,62,
283,balkanabat,TM,39.5108,54.3671,18,
313,parauapebas,BR,-6.0675,-49.9022,63,
317,remire-montjoly,GF,4.9167,-52.2667,94,
336,ghat,LY,24.9647,10.1728,38,
365,san sebastian de la gomera,ES,28.0916,-17.1133,49,
477,floriano,BR,-6.7669,-43.0225,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
balsas - nearest hotel: Imperial Hotel
san blas - nearest hotel: Suites San Blas
tan-tan - nearest hotel: Sable d´or
waddan - nearest hotel: No hotel found
balkanabat - nearest hotel: Nebitçi
parauapebas - nearest hotel: Hotel Igarapé
remire-montjoly - nearest hotel: Complexe Belova
ghat - nearest hotel: فندق تاسيلي
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
floriano - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
ipaba - nearest hotel: No hotel found
tinajo - nearest hotel: Villas Tinajo
matoury - nearest hotel: La Marmotte
paragominas - nearest hotel: No hotel found
formosa do rio preto - nearest hotel: Pousada Universo - Formosa do Rio Preto BA


,City,Country,Lat,Lng,Humidity,Hotel Name
18,balsas,BR,-7.5325,-46.0356,56,Imperial Hotel
39,san blas,MX,21.5167,-105.2667,71,Suites San Blas
192,tan-tan,MA,28.4380,-11.1032,27,Sable d´or
232,waddan,LY,29.1614,16.1390,62,No hotel found
283,balkanabat,TM,39.5108,54.3671,18,Nebitçi
313,parauapebas,BR,-6.0675,-49.9022,63,Hotel Igarapé
317,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
336,ghat,LY,24.9647,10.1728,38,فندق تاسيلي
365,san sebastian de la gomera,ES,28.0916,-17.1133,49,Hotel Villa Gomera
477,floriano,BR,-6.7669,-43.0225,62,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='Hotel Name',
    xlabel='Longitude',
    ylabel='Latitude',
    title='Cities with Hotels',
    hover_cols=['City', 'Hotel Name', 'Country'],
    geo=True,
    tiles='OSM',
    alpha=0.5,
    cmap='viridis',
    clim=(hotel_df['Hotel Name'].min(), hotel_df['Hotel Name'].max()),
    colorbar=True,
    color='City',
    #height=600,
    #width=1200,
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)